In [136]:
import numpy as np

In [144]:
class NN():
    def __init__(self, input_shape, output_shape, layer_perceptrons, learnrate):
        # seed so random numbers are the same
        np.random.seed(1)
        
        # create input layer
        self.weights = [
            2*np.random.random((input_shape[1], layer_perceptrons[0])) - 1
        ]
                
        # create hidden layers
        for i in range(1, len(layer_perceptrons), 1):
            self.weights.append(2*np.random.random((layer_perceptrons[i-1], layer_perceptrons[i])) - 1)
            
        # create output layer
        self.weights.append(2*np.random.random((layer_perceptrons[-1], output_shape[1])) - 1)
            
        # create layer outputs
        self.layer_outputs = []
        
        # set learn rate
        self.learnrate = learnrate
            
        print([x.shape for x in self.weights])
        

    def activate(self, x, deriv=False):
        if deriv:
            return 1 - (np.tanh(x))**2
        else:
            return np.tanh(x)
    
    def forward_prop(self, data):
        # compute layer outputs
        self.layer_outputs = [data]
        for i in range(len(self.weights)):
            self.layer_outputs.append(self.activate(np.dot(self.layer_outputs[i], self.weights[i])))
            
        return self.layer_outputs[-1]
    
    def train(self, data, targets, epochs=1000):
        for e in range(epochs):
            # create empty weight change matrices
            delta_weights = []
            for i in range(len(self.weights)):
                delta_weights.append(np.zeros(self.weights[i].shape))
            
            # loop through data point by point
            for x, y in zip(data, targets):
                
                ## Forward pass ##
                output = self.forward_prop(x)
                
                ## Backward pass ##
                prev_error_term = 0
                for layer in range(len(self.weights)-1, -1, -1):
                    if layer == len(self.weights)-1:
                        # Calculate error term for the output unit
                        error = y - output
                        error_term = error * self.activate(output, deriv=True)
                        
                        delta_weights[layer] += error_term*output
                    else:
                        # Calculate the error term for the hidden layers
                        error_term = prev_error_term * self.weights[layer] * self.activate(self.layer_outputs[layer], deriv=True)[:,None]
                        
                        if layer == 0: # for input layer
                            delta_weights[layer] += error_term*x[:,None]
                        else:
                            delta_weights[layer] += error_term*self.layer_outputs[layer][:,None]
                            
                    prev_error_term = error_term

            # Update weights
            for i in range(len(self.weights)):
                self.weights[i] += self.learnrate*delta_weights[i]/data.shape[0]
            
            # print MSE
            if e % (epochs / 10) == 0:
                loss = np.mean((self.forward_prop(data) - targets) ** 2)

                if 'last_loss' in locals() and last_loss < loss:
                    print("Train loss: ", loss, "  WARNING - Loss Increasing")
                else:
                    print("Train loss: ", loss)
                last_loss = loss
            
    
    
    def predict(self, X):
        return self.forward_pass(X)
    

In [145]:
input_data = np.array([[0,0,1],
                       [0,1,1],
                       [1,0,1],
                       [1,1,1]])
                
output_labels = np.array([[0],
                          [1],
                          [1],
                          [0]])

output_labels.shape

(4, 1)

In [146]:
nn = NN(input_data.shape, output_labels.shape, (3, 1), 0.005)

nn.train(input_data, output_labels)

[(3, 3), (3, 1), (1, 1)]
Train loss:  0.354939381347
Train loss:  0.370121477323   WARNING - Loss Increasing
Train loss:  0.384506422149   WARNING - Loss Increasing
Train loss:  0.397893627808   WARNING - Loss Increasing
Train loss:  0.410166071176   WARNING - Loss Increasing
Train loss:  0.421279149053   WARNING - Loss Increasing
Train loss:  0.431244524103   WARNING - Loss Increasing
Train loss:  0.440113272136   WARNING - Loss Increasing
Train loss:  0.447960984393   WARNING - Loss Increasing
Train loss:  0.454875978455   WARNING - Loss Increasing
